In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy import stats


In [2]:
# Load the data
data = pd.read_csv("Load_Calculations_Training_Dataset.csv")


In [3]:
data = data.drop(columns=['Unnamed: 0'])
data

,Projected area (sq. in),Thickness (in),Flow Length (inches),Peak Injection Pressure (psi),Pack pressures used (psi),Actual Tonnage used (tons)
0,21,0.070,7.72,18908,8550,74
1,35,0.089,6.67,23794,6736,120
2,47,0.092,3.86,19348,12263,142
3,57,0.132,10.12,15059,16380,151
4,69,0.143,12.33,14888,12026,218
...,...,...,...,...,...,...
1531,34,0.079,6.41,23779,6393,120
1532,44,0.100,4.46,19434,12451,141
1533,55,0.129,10.20,14936,16198,148
1534,68,0.152,11.71,14768,11899,220


In [4]:
data.head()

,Projected area (sq. in),Thickness (in),Flow Length (inches),Peak Injection Pressure (psi),Pack pressures used (psi),Actual Tonnage used (tons)
0,21,0.070,7.72,18908,8550,74
1,35,0.089,6.67,23794,6736,120
2,47,0.092,3.86,19348,12263,142
3,57,0.132,10.12,15059,16380,151
4,69,0.143,12.33,14888,12026,218


In [5]:
data.isnull().sum()

Projected area (sq. in)          0
Thickness (in)                   0
Flow Length (inches)             0
Peak Injection Pressure (psi)    0
Pack pressures used (psi)        0
Actual Tonnage used (tons)       0
dtype: int64

In [6]:
data.shape

(1536, 6)

In [7]:
#Separate input features (X) and output values (y)
data.dropna(inplace=True)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1:].values

In [8]:
# Normalize the input features
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

In [9]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=42)

In [10]:
# Drop rows with missing values
data.dropna(inplace=True)


In [11]:

# Define a z-score threshold for outlier removal
z_threshold = 3

# Calculate z-scores for each feature
z_scores = np.abs(stats.zscore(data))

# Find rows where any z-score is greater than the threshold
outlier_indices = np.where(z_scores > z_threshold)

# Remove outliers
data_no_outliers = data[(z_scores < z_threshold).all(axis=1)]

# Print the number of removed outliers
num_removed_outliers = len(data) - len(data_no_outliers)
print(f"Number of removed outliers: {num_removed_outliers}")

Number of removed outliers: 0


In [12]:
# Standardize the features
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [13]:
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_normalized.shape[1],)),  # Input layer
    tf.keras.layers.Dense(128, activation='relu'), # Hidden layer with 64 neurons
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])


In [14]:
# Compile the model
model.compile(loss=['mse'], optimizer='adam', metrics=['mae'])



In [15]:
# Train the model
model.fit(X_train, y_train, epochs=5000,verbose=2,validation_split=0.3)


Epoch 1/5000
24/24 - 2s - loss: 18953.9141 - mae: 128.8470 - val_loss: 19650.1465 - val_mae: 131.4199 - 2s/epoch - 65ms/step
Epoch 2/5000
24/24 - 0s - loss: 17996.6523 - mae: 125.2664 - val_loss: 17651.2422 - val_mae: 124.0598 - 96ms/epoch - 4ms/step
Epoch 3/5000
24/24 - 0s - loss: 14234.1660 - mae: 110.0665 - val_loss: 10885.3721 - val_mae: 95.1860 - 75ms/epoch - 3ms/step
Epoch 4/5000
24/24 - 0s - loss: 5668.6465 - mae: 62.3142 - val_loss: 1455.0718 - val_mae: 26.8849 - 86ms/epoch - 4ms/step
Epoch 5/5000
24/24 - 0s - loss: 829.0151 - mae: 23.8586 - val_loss: 777.0709 - val_mae: 24.6932 - 93ms/epoch - 4ms/step
Epoch 6/5000
24/24 - 0s - loss: 666.6734 - mae: 20.9201 - val_loss: 657.3082 - val_mae: 19.4956 - 104ms/epoch - 4ms/step
Epoch 7/5000
24/24 - 0s - loss: 593.5982 - mae: 18.9834 - val_loss: 589.2117 - val_mae: 18.7064 - 104ms/epoch - 4ms/step
Epoch 8/5000
24/24 - 0s - loss: 539.7083 - mae: 17.9898 - val_loss: 545.4138 - val_mae: 17.8822 - 87ms/epoch - 4ms/step
Epoch 9/5000
24/24 -

Epoch 70/5000
24/24 - 0s - loss: 34.2693 - mae: 4.8015 - val_loss: 38.5485 - val_mae: 5.0674 - 101ms/epoch - 4ms/step
Epoch 71/5000
24/24 - 0s - loss: 33.7265 - mae: 4.7866 - val_loss: 36.6867 - val_mae: 4.9958 - 101ms/epoch - 4ms/step
Epoch 72/5000
24/24 - 0s - loss: 33.9552 - mae: 4.7753 - val_loss: 37.3404 - val_mae: 4.9914 - 94ms/epoch - 4ms/step
Epoch 73/5000
24/24 - 0s - loss: 33.6063 - mae: 4.7643 - val_loss: 35.7984 - val_mae: 4.9429 - 95ms/epoch - 4ms/step
Epoch 74/5000
24/24 - 0s - loss: 32.6546 - mae: 4.7081 - val_loss: 35.4833 - val_mae: 4.9098 - 94ms/epoch - 4ms/step
Epoch 75/5000
24/24 - 0s - loss: 32.7941 - mae: 4.7035 - val_loss: 35.2820 - val_mae: 4.9300 - 87ms/epoch - 4ms/step
Epoch 76/5000
24/24 - 0s - loss: 32.4674 - mae: 4.7287 - val_loss: 34.6029 - val_mae: 4.8770 - 90ms/epoch - 4ms/step
Epoch 77/5000
24/24 - 0s - loss: 32.2059 - mae: 4.6813 - val_loss: 34.3451 - val_mae: 4.8697 - 93ms/epoch - 4ms/step
Epoch 78/5000
24/24 - 0s - loss: 31.6819 - mae: 4.6742 - val_l

Epoch 140/5000
24/24 - 0s - loss: 26.0101 - mae: 4.2930 - val_loss: 27.7533 - val_mae: 4.3797 - 98ms/epoch - 4ms/step
Epoch 141/5000
24/24 - 0s - loss: 24.8813 - mae: 4.2034 - val_loss: 25.8255 - val_mae: 4.2537 - 92ms/epoch - 4ms/step
Epoch 142/5000
24/24 - 0s - loss: 25.1706 - mae: 4.2137 - val_loss: 25.4626 - val_mae: 4.2399 - 89ms/epoch - 4ms/step
Epoch 143/5000
24/24 - 0s - loss: 25.1786 - mae: 4.2096 - val_loss: 25.8742 - val_mae: 4.2701 - 96ms/epoch - 4ms/step
Epoch 144/5000
24/24 - 0s - loss: 25.1219 - mae: 4.1908 - val_loss: 25.7402 - val_mae: 4.2506 - 103ms/epoch - 4ms/step
Epoch 145/5000
24/24 - 0s - loss: 24.9417 - mae: 4.2150 - val_loss: 25.5671 - val_mae: 4.2506 - 96ms/epoch - 4ms/step
Epoch 146/5000
24/24 - 0s - loss: 24.8290 - mae: 4.1841 - val_loss: 25.7057 - val_mae: 4.2439 - 101ms/epoch - 4ms/step
Epoch 147/5000
24/24 - 0s - loss: 24.8737 - mae: 4.1646 - val_loss: 25.0029 - val_mae: 4.1972 - 92ms/epoch - 4ms/step
Epoch 148/5000
24/24 - 0s - loss: 24.1962 - mae: 4.120

Epoch 210/5000
24/24 - 0s - loss: 20.6678 - mae: 3.8076 - val_loss: 20.3333 - val_mae: 3.7796 - 86ms/epoch - 4ms/step
Epoch 211/5000
24/24 - 0s - loss: 21.1263 - mae: 3.8033 - val_loss: 21.0035 - val_mae: 3.8339 - 87ms/epoch - 4ms/step
Epoch 212/5000
24/24 - 0s - loss: 21.0725 - mae: 3.8359 - val_loss: 20.4337 - val_mae: 3.7694 - 85ms/epoch - 4ms/step
Epoch 213/5000
24/24 - 0s - loss: 20.7303 - mae: 3.7905 - val_loss: 20.4593 - val_mae: 3.7958 - 84ms/epoch - 3ms/step
Epoch 214/5000
24/24 - 0s - loss: 20.5365 - mae: 3.7988 - val_loss: 20.6206 - val_mae: 3.8101 - 89ms/epoch - 4ms/step
Epoch 215/5000
24/24 - 0s - loss: 19.9044 - mae: 3.7437 - val_loss: 20.2875 - val_mae: 3.7470 - 87ms/epoch - 4ms/step
Epoch 216/5000
24/24 - 0s - loss: 19.9751 - mae: 3.7370 - val_loss: 20.6865 - val_mae: 3.8086 - 92ms/epoch - 4ms/step
Epoch 217/5000
24/24 - 0s - loss: 19.8732 - mae: 3.7198 - val_loss: 21.6800 - val_mae: 3.8524 - 97ms/epoch - 4ms/step
Epoch 218/5000
24/24 - 0s - loss: 20.2561 - mae: 3.7342 

Epoch 280/5000
24/24 - 0s - loss: 16.4574 - mae: 3.3853 - val_loss: 16.5484 - val_mae: 3.3876 - 76ms/epoch - 3ms/step
Epoch 281/5000
24/24 - 0s - loss: 16.5663 - mae: 3.3720 - val_loss: 16.2806 - val_mae: 3.3438 - 79ms/epoch - 3ms/step
Epoch 282/5000
24/24 - 0s - loss: 16.8104 - mae: 3.4027 - val_loss: 16.6923 - val_mae: 3.3968 - 70ms/epoch - 3ms/step
Epoch 283/5000
24/24 - 0s - loss: 16.5049 - mae: 3.3631 - val_loss: 16.8526 - val_mae: 3.3922 - 93ms/epoch - 4ms/step
Epoch 284/5000
24/24 - 0s - loss: 16.7445 - mae: 3.3678 - val_loss: 15.9549 - val_mae: 3.3274 - 89ms/epoch - 4ms/step
Epoch 285/5000
24/24 - 0s - loss: 16.1953 - mae: 3.3398 - val_loss: 16.2566 - val_mae: 3.3490 - 90ms/epoch - 4ms/step
Epoch 286/5000
24/24 - 0s - loss: 16.9538 - mae: 3.4042 - val_loss: 17.1301 - val_mae: 3.4099 - 87ms/epoch - 4ms/step
Epoch 287/5000
24/24 - 0s - loss: 16.4017 - mae: 3.3553 - val_loss: 16.3150 - val_mae: 3.3637 - 78ms/epoch - 3ms/step
Epoch 288/5000
24/24 - 0s - loss: 15.7710 - mae: 3.3066 

...

24/24 - 0s - loss: 2.0990 - mae: 1.2345 - val_loss: 2.3135 - val_mae: 1.3083 - 83ms/epoch - 3ms/step
Epoch 4710/5000
24/24 - 0s - loss: 2.0342 - mae: 1.2182 - val_loss: 2.3683 - val_mae: 1.3208 - 76ms/epoch - 3ms/step
Epoch 4711/5000
24/24 - 0s - loss: 2.5660 - mae: 1.3375 - val_loss: 2.6816 - val_mae: 1.3931 - 83ms/epoch - 3ms/step
Epoch 4712/5000
24/24 - 0s - loss: 2.4975 - mae: 1.3171 - val_loss: 2.4149 - val_mae: 1.2980 - 93ms/epoch - 4ms/step
Epoch 4713/5000
24/24 - 0s - loss: 2.2377 - mae: 1.2669 - val_loss: 2.6936 - val_mae: 1.3984 - 84ms/epoch - 3ms/step
Epoch 4714/5000
24/24 - 0s - loss: 2.0857 - mae: 1.2388 - val_loss: 2.3457 - val_mae: 1.3013 - 79ms/epoch - 3ms/step
Epoch 4715/5000
24/24 - 0s - loss: 2.1920 - mae: 1.2399 - val_loss: 2.4558 - val_mae: 1.3014 - 81ms/epoch - 3ms/step
Epoch 4716/5000
24/24 - 0s - loss: 2.1362 - mae: 1.2473 - val_loss: 2.8579 - val_mae: 1.3793 - 81ms/epoch - 3ms/step
Epoch 4717/5000
24/24 - 0s - loss: 2.2171 - mae: 1.2614 - val_loss: 2.2114 - val

24/24 - 0s - loss: 2.2935 - mae: 1.2811 - val_loss: 2.1951 - val_mae: 1.2666 - 91ms/epoch - 4ms/step
Epoch 4780/5000
24/24 - 0s - loss: 2.0207 - mae: 1.2073 - val_loss: 2.3569 - val_mae: 1.2673 - 88ms/epoch - 4ms/step
Epoch 4781/5000
24/24 - 0s - loss: 2.2628 - mae: 1.2553 - val_loss: 2.2779 - val_mae: 1.2817 - 81ms/epoch - 3ms/step
Epoch 4782/5000
24/24 - 0s - loss: 2.1768 - mae: 1.2506 - val_loss: 2.1342 - val_mae: 1.2641 - 98ms/epoch - 4ms/step
Epoch 4783/5000
24/24 - 0s - loss: 2.0528 - mae: 1.2053 - val_loss: 2.7892 - val_mae: 1.3818 - 98ms/epoch - 4ms/step
Epoch 4784/5000
24/24 - 0s - loss: 2.3006 - mae: 1.2658 - val_loss: 2.4260 - val_mae: 1.3291 - 91ms/epoch - 4ms/step
Epoch 4785/5000
24/24 - 0s - loss: 2.1788 - mae: 1.2544 - val_loss: 2.4511 - val_mae: 1.3055 - 86ms/epoch - 4ms/step
Epoch 4786/5000
24/24 - 0s - loss: 2.3874 - mae: 1.2960 - val_loss: 2.3587 - val_mae: 1.3045 - 89ms/epoch - 4ms/step
Epoch 4787/5000
24/24 - 0s - loss: 2.2196 - mae: 1.2457 - val_loss: 2.6883 - val

24/24 - 0s - loss: 2.4873 - mae: 1.3163 - val_loss: 2.3019 - val_mae: 1.2887 - 108ms/epoch - 5ms/step
Epoch 4850/5000
24/24 - 0s - loss: 2.1423 - mae: 1.2577 - val_loss: 2.1538 - val_mae: 1.2633 - 117ms/epoch - 5ms/step
Epoch 4851/5000
24/24 - 0s - loss: 2.0549 - mae: 1.2291 - val_loss: 2.3029 - val_mae: 1.3140 - 99ms/epoch - 4ms/step
Epoch 4852/5000
24/24 - 0s - loss: 2.1856 - mae: 1.2526 - val_loss: 2.6835 - val_mae: 1.3844 - 96ms/epoch - 4ms/step
Epoch 4853/5000
24/24 - 0s - loss: 2.3486 - mae: 1.2888 - val_loss: 2.2863 - val_mae: 1.2924 - 89ms/epoch - 4ms/step
Epoch 4854/5000
24/24 - 0s - loss: 2.0778 - mae: 1.2290 - val_loss: 2.2620 - val_mae: 1.2861 - 66ms/epoch - 3ms/step
Epoch 4855/5000
24/24 - 0s - loss: 2.0412 - mae: 1.2313 - val_loss: 2.1277 - val_mae: 1.2566 - 64ms/epoch - 3ms/step
Epoch 4856/5000
24/24 - 0s - loss: 2.0684 - mae: 1.2437 - val_loss: 2.5752 - val_mae: 1.3354 - 66ms/epoch - 3ms/step
Epoch 4857/5000
24/24 - 0s - loss: 2.1373 - mae: 1.2376 - val_loss: 2.7371 - v

24/24 - 0s - loss: 2.3788 - mae: 1.2799 - val_loss: 2.3006 - val_mae: 1.2674 - 76ms/epoch - 3ms/step
Epoch 4920/5000
24/24 - 0s - loss: 2.0280 - mae: 1.2194 - val_loss: 2.1621 - val_mae: 1.2429 - 78ms/epoch - 3ms/step
Epoch 4921/5000
24/24 - 0s - loss: 2.0153 - mae: 1.2124 - val_loss: 2.1272 - val_mae: 1.2479 - 90ms/epoch - 4ms/step
Epoch 4922/5000
24/24 - 0s - loss: 2.0344 - mae: 1.2246 - val_loss: 3.1605 - val_mae: 1.4814 - 104ms/epoch - 4ms/step
Epoch 4923/5000
24/24 - 0s - loss: 2.6394 - mae: 1.3501 - val_loss: 2.7778 - val_mae: 1.4109 - 82ms/epoch - 3ms/step
Epoch 4924/5000
24/24 - 0s - loss: 2.4192 - mae: 1.3062 - val_loss: 2.4078 - val_mae: 1.3167 - 81ms/epoch - 3ms/step
Epoch 4925/5000
24/24 - 0s - loss: 2.0244 - mae: 1.2153 - val_loss: 2.2508 - val_mae: 1.2890 - 70ms/epoch - 3ms/step
Epoch 4926/5000
24/24 - 0s - loss: 2.1709 - mae: 1.2541 - val_loss: 2.5721 - val_mae: 1.3256 - 81ms/epoch - 3ms/step
Epoch 4927/5000
24/24 - 0s - loss: 2.1248 - mae: 1.2249 - val_loss: 2.2544 - va

Epoch 4989/5000
24/24 - 0s - loss: 2.5490 - mae: 1.3438 - val_loss: 2.2382 - val_mae: 1.2766 - 77ms/epoch - 3ms/step
Epoch 4990/5000
24/24 - 0s - loss: 2.2458 - mae: 1.2598 - val_loss: 2.4280 - val_mae: 1.3219 - 78ms/epoch - 3ms/step
Epoch 4991/5000
24/24 - 0s - loss: 2.9020 - mae: 1.4085 - val_loss: 3.2854 - val_mae: 1.5243 - 74ms/epoch - 3ms/step
Epoch 4992/5000
24/24 - 0s - loss: 2.4334 - mae: 1.3155 - val_loss: 2.9771 - val_mae: 1.4441 - 95ms/epoch - 4ms/step
Epoch 4993/5000
24/24 - 0s - loss: 2.6464 - mae: 1.3648 - val_loss: 2.2641 - val_mae: 1.3021 - 75ms/epoch - 3ms/step
Epoch 4994/5000
24/24 - 0s - loss: 2.7393 - mae: 1.3833 - val_loss: 2.3796 - val_mae: 1.3093 - 76ms/epoch - 3ms/step
Epoch 4995/5000
24/24 - 0s - loss: 2.1325 - mae: 1.2430 - val_loss: 2.1874 - val_mae: 1.2648 - 70ms/epoch - 3ms/step
Epoch 4996/5000
24/24 - 0s - loss: 2.2288 - mae: 1.2677 - val_loss: 2.6328 - val_mae: 1.3314 - 70ms/epoch - 3ms/step
Epoch 4997/5000
24/24 - 0s - loss: 2.2372 - mae: 1.2720 - val_lo

In [16]:
# Calculating predictions for test data
y_pred= model.predict(X_test)

15/15 [==============================] - 0s 1ms/step


In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [18]:
# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")


Mean Squared Error: 2.50
Mean Absolute Error: 1.35


In [19]:
# Save the trained model
model.save("trained_model.h5")

In [20]:

# Function to predict load based on user input
def predict_load(input_features):
    input_features_normalized = scaler.transform([input_features])
    predicted_load = model.predict(input_features_normalized)
    return predicted_load[0][0]

# Get user input
user_input = []

# Collect user input for each feature
num_input_features = 5  # Replace with the number of input features
for i in range(num_input_features):
    feature_value = float(input(f"Enter value for Input Feature {i+1}: "))
    user_input.append(feature_value)

# Predict load
predicted_load = predict_load(user_input)
print(f"Predicted Load: {predicted_load:.2f}")

Enter value for Input Feature 1: 57
Enter value for Input Feature 2: 0.119
Enter value for Input Feature 3: 10.05
Enter value for Input Feature 4: 15084
Enter value for Input Feature 5: 16510
1/1 [==============================] - 0s 31ms/step
Predicted Load: 149.60
